<a href="https://colab.research.google.com/github/nehaa28/JulyTask/blob/main/part_6_UsingMLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow -q
!pip install tensorflow

     |████████████████████████████████| 14.2MB 208kB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 645kB 45.4MB/s 
     |████████████████████████████████| 1.1MB 47.3MB/s 
     |████████████████████████████████| 153kB 49.0MB/s 
     |████████████████████████████████| 174kB 43.4MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 


In [2]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
from tensorflow.keras import initializers
from tensorflow.python.keras import activations

print(tf.__version__)

# downloading fashion_mnist data
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0

test_images = test_images / 255.0              


import mlflow.tensorflow
mlflow.tensorflow.autolog()

2.5.0
4423680/4422102 [==============================] - 0s 0us/step


In [3]:
#without BatchNormalization
activation = tf.keras.activations.relu

model = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(300, activation=activation),
tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary() # model summary

with mlflow.start_run():
    model.fit(train_images, train_labels, epochs=10)
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    print('test_loss', test_loss)
    print('test_accuracy', test_acc)

    mlflow.tensorflow.mlflow.log_metric('test_loss', test_loss)
    mlflow.tensorflow.mlflow.log_metric('test_acc', test_acc)  
    mlflow.tensorflow.mlflow.log_param('initializer',model.layers[1].kernel_initializer)
    mlflow.tensorflow.mlflow.log_param('activation', activation)

del model


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3010      
Total params: 238,510
Trainable params: 238,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4787 - accuracy: 0.8299
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3614 - accuracy: 0.8699
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3220 - accuracy: 0.8824
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss

In [9]:
#with BatchNormalization
activation = tf.keras.activations.relu
initializer = tf.keras.initializers.glorot_normal
model = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dense(300, activation=activation),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary() # model summary

with mlflow.start_run():
    model.fit(train_images, train_labels, epochs=10)
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    print('test_loss', test_loss)
    print('test_accuracy', test_acc)

    mlflow.tensorflow.mlflow.log_metric('test_loss', test_loss)
    mlflow.tensorflow.mlflow.log_metric('test_acc', test_acc)  
    mlflow.tensorflow.mlflow.log_param('initializer',initializer)
    mlflow.tensorflow.mlflow.log_param('activation', activation)

del model


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 784)               3136      
_________________________________________________________________
dense_8 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_8 (Batch (None, 300)               1200      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                3010      
Total params: 242,846
Trainable params: 240,678
Non-trainable params: 2,168
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4639 - accuracy: 0.8341
Epoch 2/

In [10]:
model_A = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(128, activation=activation),
tf.keras.layers.Dense(10)
])

model_A.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model_A.summary()# model summary
model_A.fit(train_images, train_labels, epochs=10)

test_loss, test_acc = model_A.evaluate(test_images,  test_labels, verbose=2)

print('test_loss', test_loss)
print('test_accuracy', test_acc)

model_A.save('model_A.h5')

2021/07/08 16:34:56 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '871a58b73b3e4e31891c1a46ff207391', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5025 - accuracy: 0.8238
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3780 - accuracy: 0.8647
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3370 - accuracy: 0.8766
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - lo

In [11]:
# loading trained model
pretrained_mnist_model = tf.keras.models.load_model('model_A.h5')
pretrained_mnist_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [14]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]
model_B_on_A = tf.keras.models.Sequential(lower_pretrained_layers)
model_B_on_A.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model_B_on_A.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 100,609
Trainable params: 100,609
Non-trainable params: 0
_________________________________________________________________


In [15]:
loss = tf.keras.losses.binary_crossentropy
model_B_on_A.compile(loss=loss,
                  optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3),
                  metrics = ['accuracy'])

In [16]:
model_B_on_A.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 100,609
Trainable params: 100,609
Non-trainable params: 0
_________________________________________________________________


In [21]:
activation = tf.keras.activations.tanh

model = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(300, activation=activation),
tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_16 (Dense)             (None, 10)                3010      
Total params: 238,510
Trainable params: 238,510
Non-trainable params: 0
_________________________________________________________________
